# ASBD PRACTICE - PROBLEM SET 7 - MapReduce & SPARK                      

**Bhaavanaa Thumu - CED17I021**

In [ ]:
!!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.context import SparkContext
sc = SparkContext('local')

###**Problem 1**

For the given input file, calculate Wordcount using Hadoop MapReduce and Spark. Also, develop an equivalent conventional program (without spark RDDs) and compare the time taken by the two versions.

In [ ]:
text_file = sc.textFile("WordCount_input.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
#counts.saveAsTextFile("hdfs://...")
output = counts.collect()
for (word, count) in output:
        print("%s: %i" % (word, count))

Lincoln's: 1
First: 1
Inaugural: 1
Address: 1
March: 1
4,: 1
1861: 1
: 119
Fellow: 1
citizens: 5
of: 145
the: 240
United: 5
States:: 1
in: 73
compliance: 1
with: 20
a: 53
custom: 1
as: 34
old: 3
government: 10
itself,: 1
I: 42
appear: 1
before: 3
you: 15
to: 130
address: 2
briefly: 1
and: 96
take,: 1
your: 3
presence,: 1
oath: 4
prescribed: 2
by: 40
Constitution: 14
States,: 6
be: 75
taken: 1
President: 2
"before: 1
he: 1
enters: 1
on: 12
execution: 1
his: 6
office.": 1
do: 14
not: 33
consider: 2
it: 31
necessary,: 1
at: 5
present,: 1
for: 24
me: 4
discuss: 1
those: 4
matters: 1
administration: 3
about: 2
which: 16
there: 14
is: 45
no: 29
special: 1
anxiety,: 1
or: 35
excitement.: 1
Apprehension: 1
seems: 2
exist: 2
among: 6
people: 14
Southern: 1
States: 8
that: 54
accession: 1
Republican: 1
their: 14
property: 2
peace: 1
personal: 2
security: 3
are: 20
endangered.: 1
There: 4
has: 6
never: 7
been: 4
any: 28
reasonable: 2
cause: 1
such: 13
apprehension.: 1
Indeed,: 1
most: 4
ample: 1


###**Problem 2**

Consider two 10*10 matrix and perform Matrix Multiplication using Hadoop Mapreduce and Pyspark. (Note: matrix elements can be randomly populated)

In [ ]:
import numpy as np
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg.distributed import *

def as_block_matrix(rdd, rowsPerBlock=1024, colsPerBlock=1024):
    return IndexedRowMatrix(
        rdd.zipWithIndex().map(lambda xi: IndexedRow(xi[1], xi[0]))
    ).toBlockMatrix(rowsPerBlock, colsPerBlock)

A = np.arange(10 ** 2, dtype = np.float64).reshape(10, 10)
B = np.arange(10 ** 2, dtype = np.float64).reshape(10, 10)

matrixA = as_block_matrix(sc.parallelize(A))
matrixB = as_block_matrix(sc.parallelize(B))
product = matrixA.multiply(matrixB)
product

###**Problem 3**

Randomly populate 1000 numbers and calculate mean, variance, standard deviation for the generated data.

In [ ]:
import numpy as np

def func(x):
  return np.random.randint(1000)

rdd = sc.parallelize([0] * 1000).map(func)

print("rdd collect : ", rdd.collect())
print("rdd mean : ", rdd.mean())
print("rdd variance : ", rdd.variance())
print("rdd stdev : ", rdd.stdev())

rdd collect :  [340, 46, 360, 449, 625, 274, 793, 745, 273, 583, 746, 333, 158, 349, 3, 781, 672, 475, 452, 122, 73, 507, 222, 82, 183, 748, 40, 972, 663, 597, 951, 616, 40, 474, 142, 506, 988, 633, 113, 9, 602, 109, 439, 912, 987, 273, 948, 487, 187, 657, 177, 387, 364, 185, 822, 351, 426, 532, 2, 721, 372, 464, 860, 121, 32, 706, 831, 666, 725, 569, 519, 940, 987, 655, 836, 992, 357, 851, 936, 129, 146, 655, 384, 770, 446, 170, 26, 290, 997, 714, 924, 174, 240, 781, 296, 712, 515, 777, 666, 70, 941, 4, 551, 789, 38, 655, 949, 120, 966, 86, 377, 765, 774, 797, 314, 161, 726, 507, 28, 742, 567, 266, 612, 717, 86, 314, 248, 649, 814, 809, 508, 363, 20, 226, 608, 172, 16, 303, 337, 148, 641, 671, 608, 656, 780, 363, 227, 504, 3, 217, 944, 499, 302, 756, 395, 811, 332, 895, 291, 43, 342, 973, 914, 253, 890, 840, 266, 307, 49, 769, 287, 86, 785, 48, 457, 522, 794, 361, 104, 408, 468, 622, 902, 737, 927, 707, 15, 12, 4, 578, 467, 149, 340, 206, 258, 635, 273, 754, 605, 654, 810, 710, 819, 7

###**Problem 4**

Compute correlation between the given two series using Pearson’s and Spearman’s Method. (Use the Spark MLlib libraries and helper functions available)

Series A: 35, 23, 47, 17, 10, 43, 9, 6, 28

Series B: 30, 33, 45, 23, 8, 49, 12, 4, 31

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
from pyspark.mllib.stat import Statistics

seriesX = sc.parallelize([35, 23, 47, 17, 10, 43, 9, 6, 28])
seriesY = sc.parallelize([30, 33, 45, 23, 8, 49, 12, 4, 31])

print("Pearson Correlation is : " + str(Statistics.corr(seriesX, seriesY, method = "pearson")))
print("Spearman Correlation is : " + str(Statistics.corr(seriesX, seriesY, method = "spearman")))

Pearson Correlation is : 0.9498663391552045
Spearman Correlation is : 0.9


###**Problem 5**

Remove Stop wordsfrom the ‘sentence’ column given in the “StopWordRemoval_input” document.

###**Problem 6**

In the given input file, binarize the column ‘E’ (fifth column). Set the threshold as 2.5

In [ ]:
from pyspark.ml.feature import Binarizer

da1 = spark.read.csv("Binarize_input.tsv", inferSchema = True, sep = ',')

binarizer = Binarizer(threshold = 2.5, inputCol = "E", outputCol = "binarized_feature")
binarizer.getThreshold()

2.5

###**Problem 7**

a. Randomly generate 10k numbers and apply the following functions on the generated random numbers. Also, develop an equivalent conventional program (without spark RDDs) and compare the time taken by the two versions.

Exponential Function, f1(x)=ex

Logarithmic Function, f2(x)=log(x)

Square root Function, f3(x)=√x

Square function, f(4)=x2

b. Extension to the previous question: Instead of printing transformed values for all 10k words, print transformed values of the first 100 numbers only. Also, develop an equivalent conventional program (without spark RDDs) and compare the time taken by the two versions.

In [ ]:
import numpy as np
import math

def f(x):
  return np.random.randint(1000)

def func(x):
  y = math.log(2.718, x)
  return y

rdd = sc.parallelize([0] * 10000).map(f)

print("rdd.collect() : ", rdd.collect())

# exp 
print("\n2^x : ", rdd.map(lambda x: (x, math.pow(2,x))).collect())

# square root
print("\nsqrt(x) : ", rdd.map(lambda x: (x, math.sqrt(x))).collect())

# x^2
print("\nx^2 : ", rdd.map(lambda x: (x, x*x)).collect())

# logx
# print("\nlogx : ", rdd.map(lambda x: Func))

rdd.collect() :  [127, 411, 296, 313, 264, 658, 651, 795, 268, 750, 666, 736, 764, 277, 846, 407, 995, 689, 486, 369, 64, 282, 598, 877, 451, 752, 158, 896, 459, 805, 674, 158, 53, 635, 558, 378, 592, 572, 921, 773, 985, 695, 321, 561, 590, 141, 37, 941, 396, 846, 198, 749, 601, 595, 3, 207, 477, 880, 365, 174, 599, 832, 866, 617, 238, 611, 418, 877, 363, 720, 340, 172, 183, 700, 343, 284, 160, 242, 842, 480, 516, 836, 315, 692, 123, 601, 959, 261, 409, 739, 887, 537, 652, 416, 663, 551, 198, 281, 702, 538, 80, 756, 407, 715, 107, 595, 516, 63, 405, 84, 198, 17, 770, 430, 456, 777, 524, 299, 9, 651, 661, 850, 354, 663, 919, 337, 523, 579, 197, 217, 816, 396, 54, 91, 276, 648, 331, 140, 807, 384, 971, 183, 260, 486, 57, 426, 576, 819, 711, 50, 938, 30, 231, 29, 965, 319, 311, 452, 857, 561, 710, 440, 495, 517, 279, 475, 748, 211, 820, 759, 368, 164, 256, 841, 539, 104, 615, 253, 862, 38, 250, 903, 385, 708, 586, 415, 438, 291, 862, 367, 363, 12, 860, 668, 399, 232, 523, 363, 809, 553, 2